In [ ]:
!nvidia-smi  # GPU 확인

# 1. 드라이브 마운트
from google.colab import drive
drive.mount('/content/drive')

# 2. 클론할 위치로 이동
%cd /content/drive/MyDrive/Colab_Notebooks/Projects/3-2/CV

# 3. GitHub 레포 클론
!git clone https://github.com/MeshalAlamr/quick-action-recognition.git

# 4. 프로젝트 폴더로 이동
%cd quick-action-recognition

# 5. 라이브러리 설치
!pip install torch torchvision
!pip install numpy tqdm


In [ ]:
!pip install gdown

FILE_ID = "103NOL9YYZSW1hLoWmYnv5Fs8mK-Ij7qb"
ZIP_NAME = "dataset.zip"

!gdown --id {FILE_ID} -O {ZIP_NAME}
!mkdir -p data
!unzip -q {ZIP_NAME} -d data

print("다운로드 & 압축 해제 완료!")


In [ ]:
import numpy as np

path = "/content/quick-action-recognition/data/data/NTU-RGB-D/xview/"

# Load the data
data = np.load(path + "train_data.npy")

# Shape = (# of videos, coordinate axes, # of frames, # of joints, # of subjects)
shape = list(data.shape)

# Halve the number of frames
shape[2] = int(shape[2]/2)

# Initialization
downsampled_data = np.zeros(shape,dtype=np.float32)

# Downsample the data, take one frame leave another
x=0
for i in range(300):
    if i%2==0:
        downsampled_data[:,:,x,:,:] = data[:,:,i,:,:]
        x = x+1

# Save the downsampled data
np.save(path + "train_data_downsampled.npy", downsampled_data)

In [ ]:
# --- Val/Test ---
val = np.load(path + "val_data.npy")
val_ds = val[:, :, ::2, :, :]
np.save(path + "val_data_downsampled.npy", val_ds)

In [ ]:
import numpy as np
import pickle

# Selected actions (Note: 0 means action "A1" and so on)
actions = [0, 1, 7, 8, 10, 11, 27, 28, 29, 40, 102, 103]

path = "/content/quick-action-recognition/data/data/NTU-RGB-D/xview/"
# Load training and testing data and labels
train_data = np.load(path + "train_data_downsampled.npy")
test_data = np.load(path + "val_data_downsampled.npy")
with open(path + "train_label.pkl","rb") as f:
    train_label = pickle.load(f)
with open(path + "val_label.pkl","rb") as f:
    test_label = pickle.load(f)

# Find index for selected actions
train_index = [i for i, x in enumerate(train_label[1]) if x in actions]
test_index = [i for i, x in enumerate(test_label[1]) if x in actions]

# Create & save new training and testing data
small_train_data = train_data[train_index]
np.save(path + "small_train_data.npy", small_train_data)
small_test_data = test_data[test_index]
np.save(path + "small_val_data.npy", small_test_data)

# Create & save new training and testing labels
small_train_label = np.array(train_label)
small_train_label = small_train_label[:,train_index]
small_test_label = np.array(test_label)
small_test_label = small_test_label[:,test_index]

with open(path + "small_train_label.pkl", "wb") as f:
    pickle.dump(small_train_label, f)

with open(path + "small_val_label.pkl", "wb") as f:
    pickle.dump(small_test_label, f)